# Исследование надежности заемщиков

## Шаг. Обзор данных

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
# ознакомимся с данными 
df.info()
df.head(10)
#переименуем столбец "dob_years" в "full_years" для удобной работы с ним
df.rename(columns={'dob_years':'full_years'}, inplace=True)
df.head(10)
#приведем столбец 'education' к одному регистру 
df['education'] = df['education'].str.lower()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,full_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Вывод:
При изучении общей информации о данных обнаружены следующие проблемы: 
Во-первых, дни ('days_employed') не могут быть вещественным числом и не могут быть отрицательным числом. 
Во-второх, имеет смысл переименовать столбец 'dob_years' в 'full_years'.
В-третьих, столбец 'education' имеет разный регистр букв, необходимо привести к одному виду перед анализом.
Доход 'total_income' необходимо перевести в int.

## Шаг.  Заполнение пропусков

### Шаг. Проверка данных на аномалии и исправления

In [2]:
# сколько пустых значений
df.isnull().sum()
# посмотрим пустые значения(Обнаружены пустые значения NaN - замещает отсутствующее в ячейке число типа float.)
df[df['total_income'].isnull()].tail()
# проверяем какую долю составляют пропущенные значения в каждом из столбцов с пропусками.
pass_days_employed = len(df[df['days_employed'].isna()]) #сколько раз пропущен трудовой стаж
quantity_days_employed = len(df['days_employed'])
pass_total_income = len(df[df['total_income'].isna()]) #сколько раз пропущен ежемесячный доход
quantity_total_income = len(df['total_income'])
print(f'Доля пропущенных значений {pass_days_employed / quantity_days_employed:.2%}') 
print(f'Доля пропущенных значений {pass_total_income / quantity_total_income:.2%}') 
days_employed_mean = df['days_employed'].mean() #найдем ср. знач стажа 
total_income_median = df['total_income'].median() #найдем медиану дохода 
#заменим отрицательные числа на положительные в стаже
def negative_to_pozitive(value):
    if value < 0:
        value *= -1
        return value
    else:
        return value
df['days_employed'] = df['days_employed'].apply(negative_to_pozitive)
df['days_employed'] = df['days_employed'].fillna(days_employed_mean)
df['total_income'] = df['total_income'].fillna(total_income_median)

Доля пропущенных значений 10.10%
Доля пропущенных значений 10.10%


Вывод: При проверки данных обнаружены пустые значения NaN в столбцах 'days_employed' и 'total_income' (замещает отсутствующее в ячейке число типа float).
Доля пропущенных значений в каждом столбце составляет 10.10%. 
Возможная причина возникновения пропусков связаны с тем, что у людей нет подтвержденного дохода и стажа (неофицальная работа и тд).
Для несуществующих значений в столбце 'total_income' используем метод median() для получения более объективных данных т.к. некоторые значения сильно выделяются среди большинства.
В столбце 'days_employed' отрицательные числа сделали положительными, пропуски в столбце 'days_employed' заменили на среднее значение. В столбце 'total_income' пропуски заменили на медианное значение. 

### Шаг. Изменение типов данных

In [3]:
df['days_employed'] = df['days_employed'].astype('int') # заменим тип float в столбце 'days_employed' на тип int
df['total_income'] = df['total_income'].astype('int') # заменим тип float в столбце 'total_income' на тип int
df.head(10)

,children,days_employed,full_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Вывод: Значения стажа работы и доход приводим к целому числу методом astype, поскольку нужен конкретный целочисленный тип, а не вещественный для дальнейшего анализа. Приведение к целому числу также позволит нам выделять одних и тех же людей, но с разными целями по кредитам.

In [4]:
df.duplicated().sum() #поиск дубликатов 
df.drop_duplicates(subset=None, keep="first", inplace=True) #удалим дубликаты
df.duplicated().sum() #проверим наличие дубликатов после удаления 

0

Вывод:
Для поиска дубликатов в таблице используем метода duplicated т.к. при использование метода value_counts() потребуется создавать отдельный Series, который в дальнейшем не пригодится.
Дубликатов оказалось немного и их удалось быстро удалить. Возможно, было две-три одобренных заявки от одного человека либо просто человеческий фактор.

In [5]:
#проверка столбцов метод value_counts()
df['children'].value_counts() 
df['full_years'].value_counts()
df['purpose'].value_counts()
df['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

Дополнительно изучив данные на уровне подсчета уникальных значений (value_counts()) для каждого столбца, был выявлен еще ряд ошибок:
1) Столбец 'children' содержит значение 20 и -1, хотя отрицательного числа быть не может и 20 сильно выделяется на фоне остальной выборки;
Причина: Значение 20 в количестве детей скорее всего опечатка, когда набирая 2, случайно задели ноль, а значение -1 могло отнести к ошибки выгрузки данных как в ситуации со стажем работы. 
2) Столбец 'full_years' содержит нулевой возраст;
Причина: Нулевой возраст скорее всего он нам просто неизвестен. 
3) Столбец 'gender' содержит пол XNA;
Причина: Поскольку пол XNA только один, а в наших данных большинство женщины - заменим значением по умолчанию - F
4) Столбец 'purpose' содержит опечатку - ремонт жильЮ
Причина: Ошибка в самой категории на сайте или системе, исправим на F т.к. их больше.

In [6]:
#исправим значение -1 для количества детей согласно нашей гипотезы (т.е. заменим на 1)
df.loc[df['children'] == -1, 'children'] = 1
# проверим
df.loc[df['children'] == -1]['children'].count()


0

In [7]:
# исправим значения 20 для количества детей согласно нашей гипотезе
df.loc[df['children'] == 20, 'children'] = 2
# проверим
df.loc[df['children'] == 20]['children'].count()

0

In [8]:
full_years_median = df['full_years'].median()
df['full_years'] = df['full_years'].fillna(full_years_median)

In [9]:
# исправим значение пола
df.loc[df['gender'] == 'XNA', 'gender'] = 'F'
# проверим
df.loc[df['gender'] == 'XNA']['gender'].count()

0

In [10]:
# исправим опечатку в purpose
df.loc[df['purpose'] == 'ремонт жилью', 'purpose'] = 'ремонт жилья'
# проверим
df.loc[df['purpose'] == 'ремонт жилью']['purpose'].count()

0

### Шаг. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

In [11]:
# выделим словарь для education
education_new = df[['education_id', 'education']]
# удалим дубликаты из словаря
education_new = education_new.drop_duplicates().reset_index(drop=True)
# проверим содержимое
education_new.head(10)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [12]:
# выделим словарь для family_status
family_status_new = df[['family_status_id', 'family_status']]
# удалим дубликаты из словаря
family_status_new = family_status_new.drop_duplicates().reset_index(drop=True)
# проверим содержимое
family_status_new.head(10)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [13]:
# удалим столбцы education и family_status из основной таблицы
del df['education']
del df['family_status']
#посмотрим, остались ли столбцы
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   full_years        21454 non-null  int64 
 3   education_id      21454 non-null  int64 
 4   family_status_id  21454 non-null  int64 
 5   gender            21454 non-null  object
 6   income_type       21454 non-null  object
 7   debt              21454 non-null  int64 
 8   total_income      21454 non-null  int64 
 9   purpose           21454 non-null  object
dtypes: int64(7), object(3)
memory usage: 1.8+ MB


### Шаг.  Категоризация дохода

In [14]:
# напишим функцию, которая оценивает приоритет в зависимости от уровня дохода 
def make_total_income_category(total_income):
    if 0 < total_income < 30000:
        return 'E'
    if 30001 <= total_income < 50000:
        return 'D'
    if 50001 <= total_income < 200000:
        return 'C'
    if 200001 <= total_income <1000000:
        return 'B'
    return 'A'
# добавим столбец 'total_income_category'
df['total_income_category'] = df['total_income'].apply(make_total_income_category)
# проверим 
df['total_income_category'].value_counts()

C    16016
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

Вывод: Входе анализа выяснили, что самая многочисленная группа кредитополучателей является группа "С" с доходом от 50001 до 200000 руб. Самые маленькие группы кредитополучателей являются группа "Е" и "А", с наименьшим и с наибольшим доходом соответственно.

### Шаг. Категоризация целей кредита

In [15]:
from collections import Counter

building = ['недвиж', 'жиль']
wedding = ['свадьб']
car = ['автом']
education = ['образова']

df['purpose_category'] = 0

def get_category(list_name, category):
  join = '|'.join(list_name)
  index = df[df['purpose'].str.lower().str.contains(join)].index.to_list()
  for i in index:
    df.loc[i, 'purpose_category'] = category
  return df


get_category(education, 'получения образования')
get_category(building, 'операции с недвижимостью')
get_category(wedding, 'проведение свадьбы')
get_category(car, 'операции с автомобилем')

Counter(df['purpose_category'])

Counter({'операции с недвижимостью': 10811,
         'операции с автомобилем': 4306,
         'получения образования': 4013,
         'проведение свадьбы': 2324})

Вывод: В столбце 'purpose' цели названы по-разному, но имеют одинаковый смысл. С помощью функции получили 4 категории основных целей кредита. Большинство клиентов обращаются за кредитом для операций с недвижимостью, и меньше всего на оплату свадьбы.


### Ответы на вопросы

In [16]:
# функция, что посчитает нам отношение в процентах, будем использовать для ответов на все вопросы
def function(messenger):
    return str(round((messenger.sum() / messenger.count()) * 100, 2)) + '%'
# построим сводную таблицу для ответа на вопрос: Есть ли зависимость между количеством детей и возвратом кредита в срок?
data_pivot = df.pivot_table(index=['children'], values=['debt'], aggfunc=['sum','count',function])
# сортируем
data_pivot = data_pivot.sort_values(by=('function', 'debt'))
data_pivot

,sum,count,function
,debt,debt,debt
children,,,
5,0,9,0.0%
0,1063,14091,7.54%
3,27,330,8.18%
1,445,4855,9.17%
2,202,2128,9.49%
4,4,41,9.76%


Вывод: Кредитополучатели без детей реже просрочивают оплату по кредиту, чем кредитополучатели с детьми. С увеличением количества детей наблюдается увеличение количества просроченных задолженностей, хотя люди с 3 детьми чаще платят в срок чем люди с 1 ребенком. Данные не однозначные, возможно, нужна большая выборка.

In [17]:
# так как текстовые значения family_status были удалены, необходимо их вернуть методом merge()
df_with_family_status = df.merge(family_status_new, on='family_status_id', how='left')
# построим сводную таблицу для ответа на вопрос (по фрейму со значениями из словаря); 
# Есть ли зависимость между семейным положением и возвратом кредита в срок?
data_pivot = df_with_family_status.pivot_table(index=['family_status'], values=["debt"], aggfunc=['sum', 'count', function])
# сортируем
data_pivot = data_pivot.sort_values(by=('function', 'debt'))
data_pivot

,sum,count,function
,debt,debt,debt
family_status,,,
вдовец / вдова,63,959,6.57%
в разводе,85,1195,7.11%
женат / замужем,931,12339,7.55%
гражданский брак,388,4151,9.35%
Не женат / не замужем,274,2810,9.75%


Вывод: Кредитополучатели "вдовец/вдова" и клиенты "в разводе" имеют наименьший риск просрочки. Вероятно, это связано с тем, что кредит берется более осознанно. Самая "проблематичная" категория - неженатые и состоящие в гражданском браке.

In [18]:
# построим сводную таблицу для ответа на вопрос: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
data_pivot = df.pivot_table(index=['total_income_category'], values=["debt"], aggfunc=['sum', 'count', function])
# сортируем
data_pivot = data_pivot.sort_values(by=('function', 'debt'))
data_pivot

,sum,count,function
,debt,debt,debt
total_income_category,,,
D,21,350,6.0%
B,356,5041,7.06%
A,2,25,8.0%
C,1360,16016,8.49%
E,2,22,9.09%


Вывод: Кредитополучатели с доходом от 30 тыс до 50 тыс  категории "D" являются самыми надежными. У клиентов с доходом меньше 30 тыс категория "Е" самый высокий риск невозврата денежных средств. Клиенты данной категории слабо оценивают свои возможности. Обращает на себя внимание кредитполучатели с доходом от от 50 тыс до 200 тыс, риск невозврата тоже достаточно велик. 

In [19]:
# построим сводную таблицу для ответа на вопрос: Как разные цели кредита влияют на его возврат в срок?
data_pivot = df.pivot_table(index=['purpose_category'], values=["debt"], aggfunc=['sum', 'count', function])
# сортируем
data_pivot = data_pivot.sort_values(by=('function', 'debt'))
data_pivot

,sum,count,function
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,782,10811,7.23%
проведение свадьбы,186,2324,8.0%
получения образования,370,4013,9.22%
операции с автомобилем,403,4306,9.36%


Вывод: Среди 4 категорий, чаще всего платят в срок клиенты с целью кредита операции с недвижимостью. Вероятность просрочки оплаты по кредиту больше у клиентов, которым нужен кредит на операции с автомобилем и получение образования. Кредиты на свадьбу просрочивают реже - примерно, как и в случае с недвижимостью.

## Общий вывод

При анализе данных были проделаны следующие виды работ: заполнены пропущенные значения, удалены дубликаты и выделены категории для целей кредита, группировка клиентов по уровню дохода. Применив сводные таблицы, выявлены критерии влияющие на возврат кредита. 
Самый идеальный заемщик - это человек без детей, состоящий или состоявший в официальных отношениях, имеющий уровень дохода от 30 тыс до 50 тыс и берущий деньги на недвижимость.





